# Weather Lookup
Dieses Notebook zeigt, wie Wetterdaten für Tripdaten anhand von Breite, Länge und Startzeit abgefragt werden können.

In [ ]:
import pandas as pd
import requests
from datetime import datetime
import sys
from pathlib import Path
sys.path.append(str(Path('src').resolve()))
from db import open_connection


In [ ]:
conn = open_connection()
query = "SELECT trip_id, start_lat, start_lon, start_time FROM public.trips LIMIT 5;"
trips = pd.read_sql_query(query, conn)
conn.close()
trips

In [ ]:
def fetch_weather(lat, lon, timestamp):
    ts = pd.to_datetime(timestamp)
    date = ts.date()
    url = ("https://api.open-meteo.com/v1/forecast?"
           f'latitude={lat}&longitude={lon}'
           '&hourly=temperature_2m,relativehumidity_2m,precipitation,cloudcover,snowfall,windspeed_10m'
           f'&start_date={date}&end_date={date}')
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()
    data = pd.DataFrame(resp.json()['hourly'])
    data['time'] = pd.to_datetime(data['time'])
    idx = (data['time'] - ts).abs().argsort().iloc[0]
    row = data.iloc[idx]
    return {
        'temperature': float(row['temperature_2m']),
        'humidity': float(row['relativehumidity_2m']),
        'precipitation': float(row['precipitation']),
        'cloudcover': float(row['cloudcover']),
        'snowfall': float(row['snowfall']),
        'windspeed': float(row['windspeed_10m']),
    }


In [ ]:
weather = trips.apply(lambda r: pd.Series(fetch_weather(r.start_lat, r.start_lon, r.start_time)), axis=1)
trips = pd.concat([trips, weather], axis=1)
trips